In [2]:
import pandas as pd
import math

# Create a DataFrame with the provided data

data = [
    ["<=30","high", "no", "fair", "no"],
    ["<=30","high", "no", "excellent",  "no"],
    [ "31…40","high", "no", "fair", "yes"],
    [ ">40","medium", "no", "fair", "yes"],
    [">40","low", "yes", "fair",  "yes"],
    [ ">40","low", "yes", "excellent", "no"],
    [ "30...40", "low", "yes", "excellent","yes"],
    [ "<=30", "medium", "no", "fair","no"],
    ["<=30", "low", "yes", "fair", "yes"],
    [">40", "medium", "yes", "fair", "yes"],
    ["<=30", "medium", "yes", "excellent", "yes"],
    ["31...40", "medium", "no","excellent", "yes"],
    ["31...40", "high", "yes","fair", "yes"],
    [">40", "medium", "no", "excellent", "no"],
]

# Create a Pandas DataFrame
df = pd.DataFrame(data, columns=["age", "income", "student", "credit_rating", "buys_computer"])


# Calculate the entropy of the class label 'buys_computer'
def calculate_entropy(data):
    total_samples = len(data)
    if total_samples == 0:
        return 0
    positive_samples = sum(data['buys_computer'] == 'yes')
    negative_samples = sum(data['buys_computer'] == 'no')
    p_positive = positive_samples / total_samples
    p_negative = negative_samples / total_samples
    if p_positive == 0 or p_negative == 0:
        return 0
    entropy = -p_positive * math.log2(p_positive) - p_negative * math.log2(p_negative)
    return entropy

entropy_root = calculate_entropy(df)

# Calculate Information Gain for each attribute
def calculate_information_gain(data, attribute, root_entropy):
    unique_values = data[attribute].unique()
    weighted_entropy_after_split = 0
    for value in unique_values:
        subset = data[data[attribute] == value]
        subset_entropy = calculate_entropy(subset)
        weight = len(subset) / len(data)
        weighted_entropy_after_split += weight * subset_entropy
    information_gain = root_entropy - weighted_entropy_after_split
    return information_gain

# Calculate Information Gain for each attribute
root_entropy = calculate_entropy(df)
information_gains = {}
for attribute in df.columns[:-1]:  # Exclude the 'buys_computer' column
    information_gain = calculate_information_gain(df, attribute, root_entropy)
    information_gains[attribute] = information_gain

# Identify the attribute with the highest Information Gain as the root node
root_node = max(information_gains, key=information_gains.get)

print("Information Gains:")
for attribute, gain in information_gains.items():
    print(f"{attribute}: {gain}")

print(f"\nRoot Node (Highest Information Gain): {root_node}")

Information Gains:
age: 0.24674981977443933
income: 0.02922256565895487
student: 0.15183550136234159
credit_rating: 0.04812703040826949

Root Node (Highest Information Gain): age


In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Load your dataset (replace 'data.csv' with your data file)


# Separate features (X) and target variable (y)
X = df.drop(columns=['buys_computer'])
y = df['buys_computer']

# Handle categorical variables (one-hot encoding)
categorical_cols = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
X_encoded = encoder.fit_transform(X[categorical_cols])

# Impute missing values (if any)
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X_encoded)

# Combine the encoded and imputed data with non-categorical columns
X_final = pd.concat([pd.DataFrame(X_imputed), X.drop(columns=categorical_cols)], axis=1)

# Create and fit the Decision Tree model
model = DecisionTreeClassifier()
model.fit(X_final, y)

# Get the depth of the constructed tree
tree_depth = model.get_depth()

print(f"Decision Tree Depth: {tree_depth}")

Decision Tree Depth: 4
